Interpolate the observations of _Calanus Finmarchicus_ and _Calanus Holandicus_.

The fields are prepared over a region covering the Northeast Atlantic.      
Different resolutions are tested: 0.5° X 0.5°, 1° X 1° and 2° X 2°.

In [1]:
using CSV
using Dates
using DelimitedFiles 
using NCDatasets
using PyPlot
using DIVAnd
using Statistics
using PkgVersion
const plt=PyPlot

PyPlot

## Install `Cartopy`

In [2]:
using PyCall
using PyPlot
colors = PyCall.pyimport("matplotlib.colors")
ccrs = PyCall.pyimport("cartopy.crs")
cfeature = PyCall.pyimport("cartopy.feature")
mticker = PyCall.pyimport("matplotlib.ticker")
coast = cfeature.GSHHSFeature(scale="full");
cartopyticker = PyCall.pyimport("cartopy.mpl.ticker")
lon_formatter = cartopyticker.LongitudeFormatter()
lat_formatter = cartopyticker.LatitudeFormatter()
# mainproj = ccrs.PlateCarree()
cmocean = PyCall.pyimport("cmocean")

dom = [-20.5, 11.75, 41.25, 67.]
mainproj = ccrs.Mercator(central_longitude=0.5*(dom[1] + dom[2]),
    min_latitude=dom[3], max_latitude=dom[4])
#mainproj = ccrs.epsg("3035")
datacrs = ccrs.PlateCarree();

In [3]:
run_year = false
run_month = true
makeplot = false

false

### Files and directories

In [22]:
datadir = "../data/"
figdir = "../figures/L05/"
resdir = "../results/L05/"
isdir(resdir) ? @debug("already there") : mkpath(resdir)
isdir(figdir) ? @debug("already there") : mkpath(figdir)
datafile = joinpath(datadir, "MBA_CPRdata_Emodnet_21Jan22.csv")
isfile(datafile) ? @debug("already downloaded") : download("https://dox.ulg.ac.be/index.php/s/hjWKf1F3C1Pzz1r/download", datafile)
bathfile = joinpath(datadir, "gebco_30sec_16.nc")
isfile(datafile), isfile(bathfile)

(true, true)

## Grid configuration

In [23]:
Δlon = 0.5
Δlat = 0.5
longrid = dom[1]:Δlon:dom[2]
latgrid = dom[3]:Δlat:dom[4]
@info("Size of the grid: $(length(longrid)) X $(length(latgrid))")

┌ Info: Size of the grid: 65 X 52
└ @ Main In[23]:5


### Metrics

In [24]:
_, (pm, pn), (xi, yi) = DIVAnd_rectdom(longrid, latgrid);
@info(size(xi));

┌ Info: (65, 52)
└ @ Main In[24]:2


### Prepare mask

In [25]:
bx, by, b = DIVAnd.load_bath(bathfile, true, longrid, latgrid)
@show size(b)

size(b) = (65, 52)


(65, 52)

In [26]:
_, _, mask = DIVAnd.load_mask(bathfile, true, longrid, latgrid, 0.0);
# Remove Mediterranean Sea

# - Resolution = 0.1°
# mask[220:end,1:60] .= 0;

# - Resolution = 0.5°
maskindex = findall((xi .>= 0.) .& (yi .<= 47.))
mask[maskindex] .= 0.;

## Data reading
### Read the CSV files and separate by columns

In [27]:
data = readdlm(datafile, ',', skipstart=1);

lon = data[:,3]
lat = data[:,2]
year = data[:,4]
month = data[:,5]
thedates = Date.(year, month)
calanus_finmarchicus = data[:,6]
calanus_helgolandicus = data[:,7];
@info(extrema(lon));
@info(extrema(lat));

┌ Info: (-20.447, 11.703)
└ @ Main In[27]:10
┌ Info: (41.2567, 66.808)
└ @ Main In[27]:11


In [28]:
nyears = maximum(year) - minimum(year) + 1
@info(nyears)
yearcount = zeros(nyears);
@info(extrema(year));

┌ Info: 60
└ @ Main In[28]:2
┌ Info: (1959, 2018)
└ @ Main In[28]:4


## Analysis
### 1. Basic analysis: all data together

In [29]:
L = 2.5
epsilon2 = 5.

5.0

In [30]:
@time f_finmarchicus, s = DIVAndrun(mask, (pm,pn), (xi,yi), (lon, lat), 
    calanus_finmarchicus .- 0. * mean(calanus_finmarchicus), (L, L), epsilon2);
@time f_helgolandicus, s = DIVAndrun(mask, (pm,pn), (xi,yi), (lon, lat), 
    calanus_helgolandicus .- 0. * mean(calanus_helgolandicus), (L, L), epsilon2);

  0.373378 seconds (2.05 M allocations: 196.672 MiB, 27.29% gc time)
  0.305767 seconds (2.05 M allocations: 196.654 MiB, 13.50% gc time)


## Time analysis
### Yearly

In [31]:
"""
    create_nc_results(filename, lons, lats, field, speciesname)

Write a netCDF file `filename` with the coordinates `lons`, `lats` and the
heatmap `field`. `speciesname` is used for the 'title' attribute of the netCDF.

## Examples
```julia-repl
julia> create_nc_results("Bacteriastrum_interp.nc", lons, lats, field,
    "Bacteriastrum")
```
"""
function create_nc_results(filename::String, lons, lats, times, field,
                           speciesname::String="";
                           valex=-999.9,
                           varname::String = "heatmap",
                           long_name::String = "Heatmap",
						   domain::Array = [-180., 180., -90., 90.],
						   aphiaID::Int32 = 0
                           )
    Dataset(filename, "c") do ds

        # Dimensions
        ds.dim["lon"] = length(lons)
        ds.dim["lat"] = length(lats)
        ds.dim["time"] = Inf # unlimited dimension

        # Declare variables
		nccrs = defVar(ds, "crs", Int64, ())
    	nccrs.attrib["grid_mapping_name"] = "latitude_longitude"
    	nccrs.attrib["semi_major_axis"] = 6371000.0 ;
    	nccrs.attrib["inverse_flattening"] = 0 ;

        ncfield = defVar(ds, varname, Float64, ("lon", "lat"))
        ncfield.attrib["missing_value"] = Float64(valex)
        ncfield.attrib["_FillValue"] = Float64(valex)
		ncfield.attrib["units"] = "1"
        ncfield.attrib["long_name"] = long_name
		ncfield.attrib["coordinates"] = "lat lon"
		ncfield.attrib["grid_mapping"] = "crs" ;
        
        ncerror = defVar(ds, "error", Float64, ("lon", "lat"))
        ncerror.attrib["missing_value"] = Float64(valex)
        ncerror.attrib["_FillValue"] = Float64(valex)
		ncerror.attrib["units"] = "1"
        ncerror.attrib["long_name"] = "Relative error on $(long_name)"
		ncerror.attrib["coordinates"] = "lat lon"
		ncerror.attrib["grid_mapping"] = "crs" ;
        
        nctime = defVar(ds, "time", Int64, ("time",))
        nctime.attrib["missing_value"] = Float32(valex)
        nctime.attrib["units"] = "days since 1950-01-01 00:00:00"
        nctime.attrib["long_name"] = "time"
        
        nclon = defVar(ds,"lon", Float32, ("lon",))
        # nclon.attrib["missing_value"] = Float32(valex)
        nclon.attrib["_FillValue"] = Float32(valex)
        nclon.attrib["units"] = "degrees_east"
        nclon.attrib["long_name"] = "Longitude"
		nclon.attrib["standard_name"] = "longitude"
		nclon.attrib["axis"] = "X"
		nclon.attrib["reference_datum"] = "geographical coordinates, WGS84 projection"
		nclon.attrib["valid_min"] = -180.0 ;
		nclon.attrib["valid_max"] = 180.0 ;

        nclat = defVar(ds,"lat", Float32, ("lat",))
        # nclat.attrib["missing_value"] = Float32(valex)
        nclat.attrib["_FillValue"] = Float32(valex)
        nclat.attrib["units"] = "degrees_north"
		nclat.attrib["long_name"] = "Latitude"
		nclat.attrib["standard_name"] = "latitude"
		nclat.attrib["axis"] = "Y"
		nclat.attrib["reference_datum"] = "geographical coordinates, WGS84 projection"
		nclat.attrib["valid_min"] = -90.0 ;
		nclat.attrib["valid_max"] = 90.0 ;

        # Global attributes
		ds.attrib["Species_scientific_name"] = speciesname
		ds.attrib["Species_aphiaID"] = aphiaID
		ds.attrib["title"] = "$(long_name) based on presence/absence of $(speciesname)"
		ds.attrib["institution"] = "GHER - University of Liege, MBA"
		ds.attrib["source"] = "Spatial interpolation of presence/absence data"
        ds.attrib["project"] = "EMODnet Biology Phase IV"
        ds.attrib["comment"] = "Original data prepared by P. Helaouet"
        ds.attrib["data_authors"] = "Pierre Helaouet <pihe@MBA.ac.uk>"
        ds.attrib["processing_authors"] = "C. Troupin <ctroupin@uliege>, A. Barth <a.barth@uliege.be>"
		ds.attrib["publisher_name"] = "VLIZ"
		ds.attrib["publisher_url"] = "http://www.vliz.be/"
		ds.attrib["publisher_email"] = "info@vliz.be"
        ds.attrib["created"] = Dates.format(Dates.now(), "yyyy-mm-dd HH:MM:SS")
		ds.attrib["geospatial_lat_min"] = domain[3]
		ds.attrib["geospatial_lat_max"] = domain[4]
		ds.attrib["geospatial_lon_min"] = domain[1]
		ds.attrib["geospatial_lon_max"] = domain[2]
		ds.attrib["geospatial_lat_units"] = "degrees_north"
		ds.attrib["geospatial_lon_units"] = "degrees_east"
		ds.attrib["license"] = "GNU General Public License v2.0"
		ds.attrib["citation"] = "to be filled"
		ds.attrib["acknowledgement"] = "to be filled"
		ds.attrib["tool"] = "DIVAnd"
		ds.attrib["tool_version"] = "2.7.6"
		ds.attrib["tool_doi"] = "10.5281/zenodo.4306494"
		ds.attrib["language"] = "Julia 1.7.0"
		ds.attrib["Conventions"] = "CF-1.7"
		ds.attrib["netcdf_version"] = "4"
        ds.attrib["Correlation lengh"] = L
        ds.attrib["Noise-to-signal ratio"] = epsilon2

        # Define variables
        ncfield[:] = field
        nctime[:] = times
        nclon[:] = lons
        nclat[:] = lats;

    end
end;

In [32]:
function write_nc_error(filename::String, error::Array;
    valex=-999.9, varname="error")

    Dataset(filename, "a") do ds
        ds["error"][:] = error
    end
end


write_nc_error (generic function with 1 method)

#### Loop

In [33]:
run_year = true
if run_year 
    for yyyy in 1959:2018
        goodyear = findall(year .== yyyy)
        @info("Number of observations for year $(yyyy): $(length(goodyear))")

        # Analysis
        @time f_finmarchicus, s = DIVAndrun(mask, (pm,pn), (xi,yi), (lon[goodyear], lat[goodyear]), 
            calanus_finmarchicus[goodyear] .- 0. * mean(calanus_finmarchicus[goodyear]), (L, L), epsilon2);
        @time f_helgolandicus, s = DIVAndrun(mask, (pm,pn), (xi,yi), (lon[goodyear], lat[goodyear]), 
            calanus_helgolandicus[goodyear] .- 0. * mean(calanus_helgolandicus[goodyear]), (L, L), epsilon2);
        
        # Error field
        cpme = DIVAnd_cpme(mask, (pm, pn), (xi, yi), (lon[goodyear], lat[goodyear]), 
            calanus_helgolandicus[goodyear], L, epsilon2);
        
        # Figures
        #figname = joinpath(figdir, "analysis_calanus_finmarchicus_$(yyyy).jpg")
        #make_plot_analysis(longrid, latgrid, f_finmarchicus .+ 0. * mean(calanus_finmarchicus[goodyear]), 
        #    "Calanus finmarchicus analysis ($(yyyy))", figname)

        #figname = joinpath(figdir, "analysis_calanus_helgolandicus_$(yyyy).jpg")
        #make_plot_analysis(longrid, latgrid, f_helgolandicus .+ 0. * mean(calanus_helgolandicus[goodyear]), 
        #    "Calanus helgolandicus analysis ($(yyyy))", figname)
        
        #figname = joinpath(figdir, "error_calanus_helgolandicus_$(yyyy).jpg")
        #make_plot_error(longrid, latgrid, cpme, lon[goodyear], lat[goodyear], 
        #    "Relative error field", figname)
        
        # netCDF files
        create_nc_results(joinpath(resdir, "finmarchicus_$(yyyy).nc"), 
                      longrid, latgrid, (Dates.Date(yyyy, 1, 1) - Dates.Date(1950, 1, 1)).value, 
                      f_finmarchicus .+ 0. * mean(calanus_finmarchicus[goodyear]), 
                      "Calanus finmarchicus"; valex=-999.9, 
                      varname="abundance",
                      long_name="Abundance of Calanus finmarchicus",
                      domain=dom,
                      aphiaID=Int32(104464)
                   )
        
        write_nc_error(joinpath(resdir, "finmarchicus_$(yyyy).nc"), cpme)

        create_nc_results(joinpath(resdir, "helgolandicus_$(yyyy).nc"), 
                  longrid, latgrid, (Dates.Date(yyyy, 1, 1) - Dates.Date(1950, 1, 1)).value, 
                  f_helgolandicus .+ 0. * mean(calanus_helgolandicus[goodyear]), 
                  "Calanus helgolandicus"; valex=-999.9, 
                  varname="abundance",
                  long_name="Abundance of Calanus helgolandicus",
                  domain=dom,
                  aphiaID=Int32(104466)
               )
        
        write_nc_error(joinpath(resdir, "helgolandicus_$(yyyy).nc"), cpme)

    end
end

  0.019229 seconds (35.51 k allocations: 32.310 MiB)
  0.021241 seconds (35.51 k allocations: 32.310 MiB)
  0.031230 seconds (34.92 k allocations: 32.254 MiB, 37.61% gc time)
  0.018811 seconds (34.90 k allocations: 32.254 MiB)
  0.018492 seconds (35.76 k allocations: 32.324 MiB)
  0.017972 seconds (35.76 k allocations: 32.324 MiB)
  0.027462 seconds (38.34 k allocations: 32.533 MiB, 29.01% gc time)
  0.018543 seconds (38.31 k allocations: 32.533 MiB)
  0.018714 seconds (36.91 k allocations: 32.420 MiB)
  0.018272 seconds (36.92 k allocations: 32.420 MiB)
  0.021112 seconds (38.20 k allocations: 32.536 MiB)
  0.019110 seconds (38.20 k allocations: 32.536 MiB)
  0.019083 seconds (41.30 k allocations: 32.794 MiB)
  0.025001 seconds (41.32 k allocations: 32.794 MiB, 23.17% gc time)
  0.018909 seconds (36.32 k allocations: 32.363 MiB)
  0.017923 seconds (36.32 k allocations: 32.363 MiB)
  0.019395 seconds (39.92 k allocations: 32.671 MiB)
  0.025728 seconds (39.94 k allocations: 32.672 MiB

┌ Info: Number of observations for year 1959: 2364
└ @ Main In[33]:5
┌ Info: Number of observations for year 1960: 2321
└ @ Main In[33]:5
┌ Info: Number of observations for year 1961: 2382
└ @ Main In[33]:5
┌ Info: Number of observations for year 1962: 2564
└ @ Main In[33]:5
┌ Info: Number of observations for year 1963: 2465
└ @ Main In[33]:5
┌ Info: Number of observations for year 1964: 2556
└ @ Main In[33]:5
┌ Info: Number of observations for year 1965: 2777
└ @ Main In[33]:5
┌ Info: Number of observations for year 1966: 2422
└ @ Main In[33]:5
┌ Info: Number of observations for year 1967: 2679
└ @ Main In[33]:5
┌ Info: Number of observations for year 1968: 2489
└ @ Main In[33]:5
┌ Info: Number of observations for year 1969: 2863
└ @ Main In[33]:5
┌ Info: Number of observations for year 1970: 2701
└ @ Main In[33]:5
┌ Info: Number of observations for year 1971: 2964
└ @ Main In[33]:5
┌ Info: Number of observations for year 1972: 2808
└ @ Main In[33]:5
┌ Info: Number of observations for

### Monthly

In [19]:
"""
    create_nc_results_time(filename, lons, lats)

Create a new netCDF file `filename` with the coordinates `lons`, `lats` that will 
be filled with interpolated field and error field.

## Examples
```julia-repl
julia> create_nc_results_time("Bacteriastrum_interp.nc", lons, lats)
```
"""
function create_nc_results_time(filename::String, lons, lats, speciesname="";
                           valex=-999.9,
                           varname::String = "abundance",
                           long_name::String = "Interpolated abundance",
						   domain::Array = [-180., 180., -90., 90.],
						   aphiaID::Int32 = 0
                           )
    Dataset(filename, "c") do ds

        # Dimensions
        ds.dim["lon"] = length(lons)
        ds.dim["lat"] = length(lats)
        ds.dim["time"] = Inf # unlimited dimension

        # Declare variables
		nccrs = defVar(ds, "crs", Int64, ())
    	nccrs.attrib["grid_mapping_name"] = "latitude_longitude"
    	nccrs.attrib["semi_major_axis"] = 6371000.0 ;
    	nccrs.attrib["inverse_flattening"] = 0 ;

        ncfield = defVar(ds, varname, Float64, ("lon", "lat", "time"))
        ncfield.attrib["missing_value"] = Float64(valex)
        ncfield.attrib["_FillValue"] = Float64(valex)
		ncfield.attrib["units"] = "1"
        ncfield.attrib["long_name"] = long_name
		ncfield.attrib["coordinates"] = "lat lon time"
		ncfield.attrib["grid_mapping"] = "crs" ;
        
        ncerror = defVar(ds, "error", Float64, ("lon", "lat", "time"))
        ncerror.attrib["missing_value"] = Float64(valex)
        ncerror.attrib["_FillValue"] = Float64(valex)
		ncerror.attrib["units"] = "1"
        ncerror.attrib["long_name"] = "Relative error on $(long_name)"
		ncerror.attrib["coordinates"] = "lat lon time"
		ncerror.attrib["grid_mapping"] = "crs" ;
        
        nclon = defVar(ds,"lon", Float32, ("lon",))
        # nclon.attrib["missing_value"] = Float32(valex)
        nclon.attrib["_FillValue"] = Float32(valex)
        nclon.attrib["units"] = "degrees_east"
        nclon.attrib["long_name"] = "Longitude"
		nclon.attrib["standard_name"] = "longitude"
		nclon.attrib["axis"] = "X"
		nclon.attrib["reference_datum"] = "geographical coordinates, WGS84 projection"
		nclon.attrib["valid_min"] = -180.0 ;
		nclon.attrib["valid_max"] = 180.0 ;

        nclat = defVar(ds,"lat", Float32, ("lat",))
        # nclat.attrib["missing_value"] = Float32(valex)
        nclat.attrib["_FillValue"] = Float32(valex)
        nclat.attrib["units"] = "degrees_north"
		nclat.attrib["long_name"] = "Latitude"
		nclat.attrib["standard_name"] = "latitude"
		nclat.attrib["axis"] = "Y"
		nclat.attrib["reference_datum"] = "geographical coordinates, WGS84 projection"
		nclat.attrib["valid_min"] = -90.0 ;
		nclat.attrib["valid_max"] = 90.0 ;
        
        nctime = defVar(ds,"time", Float32, ("time",))
        #nctime.attrib["_FillValue"] = Float32(valex)
        nctime.attrib["units"] = "degrees_north"
		nctime.attrib["long_name"] = "Time"
		nctime.attrib["standard_name"] = "time"
        nctime.attrib["units"] = "days since 1950-01-01 00:00:00"
		nctime.attrib["axis"] = "T"

        # Global attributes
		ds.attrib["Species_scientific_name"] = speciesname
		ds.attrib["Species_aphiaID"] = aphiaID
		ds.attrib["title"] = "$(long_name) based on presence/absence of $(speciesname)"
		ds.attrib["institution"] = "GHER - University of Liege, MBA"
		ds.attrib["source"] = "Spatial interpolation of presence/absence data"
        ds.attrib["project"] = "EMODnet Biology Phase IV"
        ds.attrib["comment"] = "Original data prepared by P. Helaouet"
        ds.attrib["data_authors"] = "Pierre Helaouet <pihe@MBA.ac.uk>"
        ds.attrib["processing_authors"] = "C. Troupin <ctroupin@uliege>, A. Barth <a.barth@uliege.be>"
		ds.attrib["publisher_name"] = "VLIZ"
		ds.attrib["publisher_url"] = "http://www.vliz.be/"
		ds.attrib["publisher_email"] = "info@vliz.be"
        ds.attrib["created"] = Dates.format(Dates.now(), "yyyy-mm-dd HH:MM:SS")
		ds.attrib["geospatial_lat_min"] = domain[3]
		ds.attrib["geospatial_lat_max"] = domain[4]
		ds.attrib["geospatial_lon_min"] = domain[1]
		ds.attrib["geospatial_lon_max"] = domain[2]
		ds.attrib["geospatial_lat_units"] = "degrees_north"
		ds.attrib["geospatial_lon_units"] = "degrees_east"
		ds.attrib["license"] = "GNU General Public License v2.0"
		ds.attrib["citation"] = "to be filled"
		ds.attrib["acknowledgement"] = "to be filled"
		ds.attrib["tool"] = "DIVAnd"
		ds.attrib["tool_version"] = "2.7.6"
		ds.attrib["tool_doi"] = "10.5281/zenodo.4306494"
		ds.attrib["language"] = "Julia 1.7.0"
		ds.attrib["Conventions"] = "CF-1.7"
		ds.attrib["netcdf_version"] = "4"
        ds.attrib["Correlation length (degrees)"] = L
        ds.attrib["Noise-to-signal ratio"] = epsilon2

        # Define variables
        nclon[:] = collect(lons)
        nclat[:] = collect(lats);

    end
end;

In [20]:
resfile1 = joinpath(resdir, "calanus_finmarchicus_year_month.nc")
resfile2 = joinpath(resdir, "calanus_helgolandicus_year_month.nc")


create_nc_results_time(resfile1, longrid, latgrid,  
                      "Calanus finmarchicus"; valex=-999.9, 
                      varname="abundance",
                      long_name="Abundance of Calanus finmarchicus",
                      domain=dom,
                      aphiaID=Int32(104464));

create_nc_results_time(resfile2, longrid, latgrid,  
                      "Calanus helgolandicus"; valex=-999.9, 
                      varname="abundance",
                      long_name="Abundance of Calanus helgolandicus",
                      domain=dom,
                      aphiaID=Int32(104466));

In [21]:
iii = 0
for yyyy in 1959:2018

    for m in 1:12
        
        iii += 1

        mm = lpad(string(m), 2, '0')
        goodtime = findall( (year .== yyyy) .& (month .== m) )
        @info("Found $(length(goodtime)) dates points for year $(yyyy) | month $(mm)")
        
        if length(goodtime) > 0

            @time f_finmarchicus, s = DIVAndrun(mask, (pm,pn), (xi,yi), (lon[goodtime], lat[goodtime]), 
                calanus_finmarchicus[goodtime] .- mean(calanus_finmarchicus[goodtime]), (L, L), epsilon2);
            @time f_helgolandicus, s = DIVAndrun(mask, (pm,pn), (xi,yi), (lon[goodtime], lat[goodtime]), 
                calanus_helgolandicus[goodtime] .- mean(calanus_helgolandicus[goodtime]), (L, L), epsilon2);
            
            # Error field
            cpme_fin = DIVAnd_cpme(mask, (pm, pn), (xi, yi), (lon[goodtime], lat[goodtime]), 
                calanus_finmarchicus[goodtime], L, epsilon2);
            cpme_hel = DIVAnd_cpme(mask, (pm, pn), (xi, yi), (lon[goodtime], lat[goodtime]), 
                calanus_helgolandicus[goodtime], L, epsilon2);
            

            figname = joinpath(figdir, "analysis_calanus_finmarchicus_$(yyyy)$(mm).jpg")
            #make_plot_analysis(longrid, latgrid, f_finmarchicus .+ mean(calanus_finmarchicus[goodtime]), 
            #    "Calanus finmarchicus analysis ($(Dates.monthname(m)) $(yyyy))", figname)

            figname = joinpath(figdir, "analysis_calanus_helgolandicus_$(yyyy)$(mm).jpg")
            #make_plot_analysis(longrid, latgrid, f_helgolandicus .+ mean(calanus_helgolandicus[goodtime]), 
            #    "Calanus finmarchicus analysis ($(Dates.monthname(m)) $(yyyy))", figname)

         
            Dataset(resfile1, "a") do ds
                ds["time"][iii] = (Dates.Date(yyyy, m, 1) - Dates.Date(1950, 1, 1)).value
                ds["error"][:,:,iii] = cpme_fin
                ds["abundance"][:,:,iii] = f_finmarchicus
            end
            
            Dataset(resfile2, "a") do ds
                ds["time"][iii] = (Dates.Date(yyyy, m, 1) - Dates.Date(1950, 1, 1)).value
                ds["error"][:,:,iii] = cpme_hel
                ds["abundance"][:,:,iii] = f_helgolandicus
            end
            
        else
            @warn("No data to perform analysis")
        end

    end

end

  0.172127 seconds (4.71 k allocations: 118.531 MiB, 4.40% gc time)
  0.236978 seconds (4.71 k allocations: 118.531 MiB, 2.85% gc time)
  0.150832 seconds (4.75 k allocations: 118.535 MiB, 4.82% gc time)
  0.181178 seconds (4.75 k allocations: 118.535 MiB, 3.63% gc time)
  0.164147 seconds (6.09 k allocations: 118.682 MiB)
  0.195356 seconds (6.10 k allocations: 118.683 MiB, 2.76% gc time)
  0.164610 seconds (4.49 k allocations: 118.499 MiB, 2.20% gc time)
  0.208539 seconds (4.49 k allocations: 118.499 MiB)
  0.160840 seconds (5.50 k allocations: 118.620 MiB)
  0.188646 seconds (5.51 k allocations: 118.620 MiB, 3.27% gc time)
  0.164253 seconds (5.84 k allocations: 118.658 MiB, 2.40% gc time)
  0.183162 seconds (5.83 k allocations: 118.658 MiB)
  0.153352 seconds (5.69 k allocations: 118.637 MiB)
  0.348805 seconds (5.70 k allocations: 118.638 MiB, 57.04% gc time)
  0.156915 seconds (5.74 k allocations: 118.648 MiB, 5.45% gc time)
  0.199297 seconds (5.74 k allocations: 118.648 MiB, 1

┌ Info: Found 141 dates points for year 1959 | month 01
└ @ Main In[21]:10
┌ Info: Found 144 dates points for year 1959 | month 02
└ @ Main In[21]:10
┌ Info: Found 247 dates points for year 1959 | month 03
└ @ Main In[21]:10
┌ Info: Found 124 dates points for year 1959 | month 04
└ @ Main In[21]:10
┌ Info: Found 202 dates points for year 1959 | month 05
└ @ Main In[21]:10
┌ Info: Found 227 dates points for year 1959 | month 06
└ @ Main In[21]:10
┌ Info: Found 216 dates points for year 1959 | month 07
└ @ Main In[21]:10
┌ Info: Found 220 dates points for year 1959 | month 08
└ @ Main In[21]:10
┌ Info: Found 239 dates points for year 1959 | month 09
└ @ Main In[21]:10
┌ Info: Found 270 dates points for year 1959 | month 10
└ @ Main In[21]:10
┌ Info: Found 183 dates points for year 1959 | month 11
└ @ Main In[21]:10
┌ Info: Found 151 dates points for year 1959 | month 12
└ @ Main In[21]:10
┌ Info: Found 160 dates points for year 1960 | month 01
└ @ Main In[21]:10
┌ Info: Found 186 dates p

  0.208556 seconds (6.79 k allocations: 118.815 MiB, 1.39% gc time)
  0.250167 seconds (6.60 k allocations: 118.794 MiB, 2.29% gc time)
  0.250499 seconds (6.60 k allocations: 118.794 MiB, 1.45% gc time)
  0.333984 seconds (6.53 k allocations: 118.787 MiB, 1.62% gc time)
  0.344306 seconds (6.52 k allocations: 118.786 MiB, 1.22% gc time)
  0.294083 seconds (5.89 k allocations: 118.663 MiB, 1.56% gc time)
  0.291924 seconds (5.88 k allocations: 118.663 MiB)
  0.245872 seconds (4.89 k allocations: 118.552 MiB)
  0.311886 seconds (4.90 k allocations: 118.552 MiB, 24.84% gc time)
  0.485878 seconds (5.08 k allocations: 118.572 MiB, 43.71% gc time)
  0.217743 seconds (5.08 k allocations: 118.572 MiB)
  0.156925 seconds (6.04 k allocations: 118.679 MiB, 2.98% gc time)
  0.165540 seconds (6.04 k allocations: 118.679 MiB, 2.08% gc time)
  0.136911 seconds (4.95 k allocations: 118.558 MiB, 2.53% gc time)
  0.172854 seconds (4.95 k allocations: 118.558 MiB, 1.21% gc time)
  0.184180 seconds (5.7

  0.182677 seconds (5.90 k allocations: 118.663 MiB, 2.20% gc time)
  0.212692 seconds (5.90 k allocations: 118.663 MiB)
  0.197880 seconds (5.83 k allocations: 118.653 MiB, 3.96% gc time)
  0.293721 seconds (5.82 k allocations: 118.653 MiB, 18.57% gc time)
  0.168628 seconds (5.54 k allocations: 118.620 MiB)
  0.353144 seconds (5.55 k allocations: 118.620 MiB, 53.89% gc time)
  0.176259 seconds (6.23 k allocations: 118.753 MiB, 6.95% gc time)
  0.203165 seconds (6.23 k allocations: 118.753 MiB, 1.85% gc time)
  0.167005 seconds (6.16 k allocations: 118.691 MiB, 2.94% gc time)
  0.405214 seconds (6.16 k allocations: 118.691 MiB, 0.69% gc time)
  0.148760 seconds (5.90 k allocations: 118.665 MiB)
  0.264998 seconds (5.90 k allocations: 118.665 MiB, 1.34% gc time)
  0.149846 seconds (5.31 k allocations: 118.597 MiB, 2.39% gc time)
  0.139727 seconds (5.31 k allocations: 118.597 MiB)
  0.181522 seconds (6.34 k allocations: 118.758 MiB, 2.12% gc time)
  0.209201 seconds (6.33 k allocations

┌ Info: Found 287 dates points for year 1969 | month 11
└ @ Main In[21]:10
┌ Info: Found 147 dates points for year 1969 | month 12
└ @ Main In[21]:10
┌ Info: Found 173 dates points for year 1970 | month 01
└ @ Main In[21]:10
┌ Info: Found 189 dates points for year 1970 | month 02
└ @ Main In[21]:10
┌ Info: Found 186 dates points for year 1970 | month 03
└ @ Main In[21]:10
┌ Info: Found 312 dates points for year 1970 | month 04
└ @ Main In[21]:10
┌ Info: Found 245 dates points for year 1970 | month 05
└ @ Main In[21]:10
┌ Info: Found 257 dates points for year 1970 | month 06
└ @ Main In[21]:10
┌ Info: Found 202 dates points for year 1970 | month 07
└ @ Main In[21]:10
┌ Info: Found 272 dates points for year 1970 | month 08
└ @ Main In[21]:10
┌ Info: Found 250 dates points for year 1970 | month 09
└ @ Main In[21]:10
┌ Info: Found 147 dates points for year 1970 | month 10
└ @ Main In[21]:10
┌ Info: Found 217 dates points for year 1970 | month 11
└ @ Main In[21]:10
┌ Info: Found 251 dates p

...

  0.133618 seconds (5.00 k allocations: 118.564 MiB, 4.93% gc time)
  0.173235 seconds (5.00 k allocations: 118.563 MiB, 1.93% gc time)
  0.128182 seconds (5.47 k allocations: 118.615 MiB, 3.57% gc time)
  0.155087 seconds (5.47 k allocations: 118.615 MiB, 1.91% gc time)
  0.133317 seconds (5.65 k allocations: 118.635 MiB, 2.71% gc time)
  0.164052 seconds (5.65 k allocations: 118.635 MiB)
  0.132951 seconds (5.81 k allocations: 118.653 MiB, 4.97% gc time)
  0.163543 seconds (5.81 k allocations: 118.653 MiB, 2.16% gc time)
  0.128825 seconds (5.22 k allocations: 118.590 MiB)
  0.166719 seconds (5.22 k allocations: 118.590 MiB, 4.44% gc time)
  0.313742 seconds (5.26 k allocations: 118.590 MiB, 56.86% gc time)
  0.159722 seconds (5.24 k allocations: 118.589 MiB)
  0.197110 seconds (5.37 k allocations: 118.603 MiB, 3.93% gc time)
  0.169071 seconds (5.36 k allocations: 118.603 MiB, 3.12% gc time)
  0.149804 seconds (5.46 k allocations: 118.615 MiB, 3.41% gc time)
  0.170865 seconds (5.45

  0.143939 seconds (6.09 k allocations: 118.684 MiB, 2.75% gc time)
  0.203547 seconds (6.09 k allocations: 118.684 MiB, 1.64% gc time)
  0.175139 seconds (5.43 k allocations: 118.608 MiB, 5.10% gc time)
  0.288689 seconds (5.43 k allocations: 118.608 MiB, 1.21% gc time)
  0.139693 seconds (5.31 k allocations: 118.598 MiB)
  0.213008 seconds (5.31 k allocations: 118.598 MiB, 3.75% gc time)
  0.179496 seconds (5.65 k allocations: 118.635 MiB, 2.74% gc time)
  0.190915 seconds (5.65 k allocations: 118.635 MiB, 1.91% gc time)
  0.154942 seconds (5.82 k allocations: 118.657 MiB, 4.56% gc time)
  0.202902 seconds (5.81 k allocations: 118.657 MiB)
  0.186400 seconds (5.94 k allocations: 118.668 MiB, 2.48% gc time)
  0.187011 seconds (5.93 k allocations: 118.667 MiB)
  0.142401 seconds (5.91 k allocations: 118.665 MiB)
  0.254047 seconds (5.91 k allocations: 118.665 MiB, 1.64% gc time)
  0.167458 seconds (6.21 k allocations: 118.746 MiB)
  0.375573 seconds (6.24 k allocations: 118.747 MiB, 57

┌ Info: Found 207 dates points for year 2002 | month 05
└ @ Main In[21]:10
┌ Info: Found 176 dates points for year 2002 | month 06
└ @ Main In[21]:10
┌ Info: Found 222 dates points for year 2002 | month 07
└ @ Main In[21]:10
┌ Info: Found 198 dates points for year 2002 | month 08
└ @ Main In[21]:10
┌ Info: Found 245 dates points for year 2002 | month 09
└ @ Main In[21]:10
┌ Info: Found 229 dates points for year 2002 | month 10
└ @ Main In[21]:10
┌ Info: Found 271 dates points for year 2002 | month 11
└ @ Main In[21]:10
┌ Info: Found 246 dates points for year 2002 | month 12
└ @ Main In[21]:10
┌ Info: Found 144 dates points for year 2003 | month 01
└ @ Main In[21]:10
┌ Info: Found 224 dates points for year 2003 | month 02
└ @ Main In[21]:10
┌ Info: Found 231 dates points for year 2003 | month 03
└ @ Main In[21]:10
┌ Info: Found 245 dates points for year 2003 | month 04
└ @ Main In[21]:10
┌ Info: Found 189 dates points for year 2003 | month 05
└ @ Main In[21]:10
┌ Info: Found 224 dates p

  0.192627 seconds (4.90 k allocations: 118.553 MiB, 1.78% gc time)
  0.171781 seconds (5.99 k allocations: 118.668 MiB, 2.56% gc time)
  0.202321 seconds (5.99 k allocations: 118.668 MiB, 1.72% gc time)
  0.169599 seconds (6.45 k allocations: 118.773 MiB, 2.50% gc time)
  0.204381 seconds (6.45 k allocations: 118.773 MiB)
  0.173016 seconds (4.95 k allocations: 118.555 MiB)
  0.203659 seconds (4.95 k allocations: 118.555 MiB, 3.35% gc time)
  0.184383 seconds (5.94 k allocations: 118.666 MiB, 2.01% gc time)
  0.190327 seconds (5.93 k allocations: 118.666 MiB)
  0.179601 seconds (5.32 k allocations: 118.598 MiB)
  0.202000 seconds (5.33 k allocations: 118.598 MiB, 2.07% gc time)
  0.110089 seconds (5.72 k allocations: 118.643 MiB)
  0.240340 seconds (5.73 k allocations: 118.643 MiB, 25.53% gc time)
  0.159993 seconds (5.41 k allocations: 118.607 MiB)
  0.353538 seconds (5.42 k allocations: 118.607 MiB, 50.71% gc time)
  0.171558 seconds (5.65 k allocations: 118.635 MiB, 2.07% gc time)


┌ Info: Found 223 dates points for year 2013 | month 03
└ @ Main In[21]:10
┌ Info: Found 208 dates points for year 2013 | month 04
└ @ Main In[21]:10
┌ Info: Found 269 dates points for year 2013 | month 05
└ @ Main In[21]:10
┌ Info: Found 176 dates points for year 2013 | month 06
└ @ Main In[21]:10
┌ Info: Found 256 dates points for year 2013 | month 07
└ @ Main In[21]:10
┌ Info: Found 234 dates points for year 2013 | month 08
└ @ Main In[21]:10
┌ Info: Found 204 dates points for year 2013 | month 09
└ @ Main In[21]:10
┌ Info: Found 279 dates points for year 2013 | month 10
└ @ Main In[21]:10
┌ Info: Found 275 dates points for year 2013 | month 11
└ @ Main In[21]:10
┌ Info: Found 275 dates points for year 2013 | month 12
└ @ Main In[21]:10
┌ Info: Found 242 dates points for year 2014 | month 01
└ @ Main In[21]:10
┌ Info: Found 245 dates points for year 2014 | month 02
└ @ Main In[21]:10
┌ Info: Found 248 dates points for year 2014 | month 03
└ @ Main In[21]:10
┌ Info: Found 191 dates p